# Monomer Potential Fields

This script demonstrates how to compute the Helmholtz free energy expressed in the monomer potential fields (such as $w_A({\bf r})$, $w_B({\bf r})$, and $w_C({\bf r})$) with the incompressible model.

Reference: [(2017) Accelerating self-consistent field theory of block polymers in a variable unit cell](http://dx.doi.org/10.1063/1.4986643)

### 1. The Helmholtz free energy (SCFT)

* The Helmholtz free energy $F$ is written as in 'per monomer' unit:
\begin{align}
\beta F &= -\sum_i n_i\log{Q_i} +
\rho_0\int d{\bf r}\left[\frac{1}{2} \sum_{ij} \chi_{ij}\phi_{i}({\bf r})\phi_{j}({\bf r})
-\sum_{i} w_{i}({\bf r})\phi_{i}({\bf r}) \right],  \ \ \ \ (\textrm{per monomer unit})
\end{align}

* The Helmholtz free energy  $F$ can be rewritten:
\begin{align}
Nw_{i} &\rightarrow w_{i}, \\
\frac{\beta F}{CV} &= \frac{\beta F}{n} = -\sum_i \frac{\bar{\phi}_i}{\alpha_i}\log{Q_i} +
\frac{1}{V}\int d{\bf r}\left[\frac{1}{2} \sum_{ij} \chi_{ij}N\phi_{i}({\bf r})\phi_{\beta}({\bf r})
-\sum_{i} w_{i}({\bf r})\phi_{i}({\bf r}) \right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The Helmholtz free energy per $p$-chain:
\begin{align}
\frac{\beta F}{n_p} &= \frac{\alpha_p}{\bar{\phi}_p}\frac{\beta F}{CV}  \ \ \ \ (\textrm{per $p$-chain})
\end{align}

* The self-consistent conditions for the potential fields:
\begin{align}
w_i({\bf r}) = \sum_{i \neq j} \chi_{ij} N \phi_j({\bf r}) + \xi({\bf r}).  \ \ \ \ (\textrm{per chain unit})
\end{align}

* ${\bf R}_w$ are field residuals:
\begin{align}
{\bf R}_w = X{\boldsymbol\phi} − P{\bf w},
\end{align}
where $X$ is $\chi$N matrix, ${\boldsymbol\phi}$ and ${\bf w}$ are vectors of $\phi$ and $w$. 

* $P$ is a matrix for computing field residuals:
\begin{align}
P = I − \frac{{\bf e}{\bf e}^TX^{-1}}{{\bf e}^TX^{-1}{\bf e}}.
\end{align}

#### Example 1) AB-type polymeric systems

* The effective Hamiltonian $H$ is written as:
\begin{align}
\beta H &= -\sum_i n_i\log{Q_i} +
\rho_0\int d{\bf r}\left[\chi_{AB}\phi_{A}({\bf r})\phi_{B}({\bf r}) - w_{A}({\bf r})\phi_{A} - w_{B}({\bf r})\phi_{B}\right],  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta H}{CV} &= -\sum_i \frac{\bar{\phi}_i}{\alpha_i}\log{Q_i} +
\frac{1}{V}\int d{\bf r}\left[\chi_{AB}N \phi_{A}({\bf r})\phi_{B}({\bf r}) - w_{A}({\bf r})\phi_{A} - w_{B}({\bf r})\phi_{B}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The self-consistent conditions for the potential fields:
\begin{align}
w_A({\bf r}) &= \chi_{AB} N \phi_B({\bf r}) + \xi({\bf r}),  \ \ \ \ (\textrm{per chain unit}) \\
w_B({\bf r}) &= \chi_{AB} N \phi_A({\bf r}) + \xi({\bf r}).  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

\begin{align}
{\bf R}_w =
\left[\begin{array}{cc} 
0 & \chi N \\ 
\chi N & 0
\end{array}\right]
\left[\begin{array}{cc} 
\phi_A({\bf r}) \\ 
\phi_B({\bf r}) 
\end{array}\right] −
\left[\begin{array}{cc} 
\ \ 1/2 & -1/2 \\ 
-1/2 & \ \ 1/2
\end{array}\right]
\left[\begin{array}{cc} 
w_A({\bf r}) \\ 
w_B({\bf r}) 
\end{array}\right]
,
\end{align}

### 2. Setting simulation parameters

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Simulation parameters
nx = [2,2]                         # grid number
lx = [2.0,2.0]                     # box size
stat_seg_lengths = {"A":1.0,       # statistical segment lengths
                    "B":1.0}        
ds = 0.01                          # contour step interval
chi_n = {"A,B":20}
monomer_types = ["A", "B"]  # monomer types

### 3. Initial potential fields and concentrations

In [17]:
# Potential fields
w_A = np.array([0.5488135, 0.71518937, 0.60276338, 0.54488318])
w_B = np.array([0.4236548, 0.64589411, 0.43758721, 0.89177300])
w = {"A": w_A, "B": w_B}  

# Concentrations (assumed that they are calculated for given potential fields)
phi_A = np.array([0.53203648, 0.47190149, 0.51749752, 0.47856451])
phi_B = np.array([0.53940409, 0.47804036, 0.53185577, 0.45069978])
phi = {"A": phi_A, "B": phi_B}

# print("w_A:", w["A"])
# print("w_B:", w["B"])
# print("Concentration A:", phi["A"])
# print("Concentration B:", phi["B"])

### 4. Computing P and field residuals
##### Example 1) AB-type polymeric systems 

In [18]:
# The number of components
M = len(monomer_types)

# X (χN matrix) and its inverse
matrix_chi = np.array([
    [0, chi_n["A,B"]],
    [chi_n["A,B"], 0]]
)
matrix_chi_inv = np.linalg.inv(matrix_chi)

# P matrix for field residuals
matrix_p = np.identity(M) - np.matmul(np.ones((M,M)), matrix_chi_inv)/np.sum(matrix_chi_inv)

# Calculate self-consistency error
w_residual = np.zeros([M, np.prod(nx)], dtype=np.float64) # array for output fields
for i in range(M):
    for j in range(M):
        w_residual[i,:] += matrix_chi[i,j]*phi[monomer_types[j]] - matrix_p[i,j]*w[monomer_types[j]]

print("X (χN matrix):\n\t", str(matrix_chi).replace("\n", "\n\t"))
print("Inverse of X:\n\t", str(matrix_chi).replace("\n", "\n\t"))
print("P matrix for field residuals:\n\t", str(matrix_p).replace("\n", "\n\t"))

print("Field residual for w_A:\n\t", w_residual[0])
print("χN phi_B + 0.5*(w_A-w_B) :\n\t", chi_n["A,B"]* phi["B"]-0.5*(w["A"]-w["B"]))

print("Field residual for w_B:\n\t", w_residual[1])
print("χN phi_A + 0.5(w_A-w_B):\n\t", chi_n["A,B"]* phi["A"]+0.5*(w["A"]-w["B"]))


X (χN matrix):
	 [[ 0 20]
	 [20  0]]
Inverse of X:
	 [[ 0 20]
	 [20  0]]
P matrix for field residuals:
	 [[ 0.5 -0.5]
	 [-0.5  0.5]]
Field residual for w_A:
	 [10.72550245  9.52615957 10.55452731  9.18744051]
χN phi_B + 0.5*(w_A-w_B) :
	 [10.72550245  9.52615957 10.55452731  9.18744051]
Field residual for w_B:
	 [10.70330895  9.47267743 10.43253849  9.39784529]
χN phi_A + 0.5(w_A-w_B):
	 [10.70330895  9.47267743 10.43253849  9.39784529]
